#### Generates Weight Sheets

In [2]:
import pandas as pd
import pdfkit
import json
import base64
import os

# Read the Excel file without skipping rows
df = pd.read_excel('Lista de Inscritos Campeonato Zonal Cadetes 2024.xlsx', header=None)

# Names for the output PDFs
tournament_code = "TESTS"

# Filter out rows with NaN values in the first column
df = df.dropna(subset=[0])

# Filter out rows starting with "Masculinos" and "Femininos"
df = df[~df[0].astype(str).str.startswith(('Masculinos', 'Femininos', 'N.º'))]

# Drop the first column
df = df.iloc[:, 1:]

# Set new column names
df.columns = ['NAME', 'GRAD', 'FED_LIC', 'WGT_CAT', 'CLUB', 'ASSOC']

# Display the filtered DataFrame
pd.set_option('display.max_rows', None)
#print(df)

In [3]:
# Determine Gender based on Weight Category
def determine_gender(row):
    masculine_categories = [-38, -42, -46, -50, -55, -60, -66, -73, -81, +81, -90, +90, -100, +100]
    if row['WGT_CAT'] in masculine_categories:
        return 'MASC'
    else:
        return 'FEM'
    
df['GENDER'] = df.apply(determine_gender, axis=1)

# Insert the 'GENDER' column as the second column
df.insert(1, 'GENDER', df.pop('GENDER'))

#df

In [4]:
# Create separate DataFrames for masculine and feminine athletes
masculine_df = df[df['GENDER'] == 'MASC']
feminine_df = df[df['GENDER'] == 'FEM']

# Create a dictionary to store athletes for each club
def create_club_dict(athletes):
    club_athletes = {}
    for club in athletes['CLUB'].unique():
        club_athletes[club] = athletes[athletes['CLUB'] == club]
    return club_athletes

masculine_club_athletes = create_club_dict(masculine_df)
feminine_club_athletes = create_club_dict(feminine_df)

In [5]:
# Load the JSON data
with open('clubs.json') as f:
    clubs_data = json.load(f)

# Create a dictionary from the JSON data
clubs_dict = {club['initials']: {'full_name': club['full_name'], 'image': club['image']} for club in clubs_data['clubs']}
clubs_dict

{'CJTN': {'full_name': 'Clube Judo Torres Novas',
  'image': 'imgClubs/clube_judo_torres_novas.jpg'},
 'CPRM': {'full_name': 'Casa Povo Rio Maior',
  'image': 'imgClubs/casa_povo_rio_maior.jpg'},
 'SFGP': {'full_name': 'Sociedade Filarmónica Gualdim Pais',
  'image': 'imgClubs/sociedade_filarmonica_gualdim_pais.jpg'},
 'JCJuncal': {'full_name': 'Judo Clube Juncal',
  'image': 'imgClubs/judo_clube_juncal.jpg'},
 'AAC': {'full_name': 'Associação Académica Coimbra',
  'image': 'imgClubs/associacao_academica_coimbra.jpg'},
 'JCMG': {'full_name': 'Judo Clube Marinha Grande',
  'image': 'imgClubs/judo_clube_marinha_grande.jpg'},
 'ACR Comeira': {'full_name': 'Associação Cultural e Recreativa Comeira',
  'image': 'imgClubs/associacao_cultural_recreativa_comeira.jpg'},
 'CBS': {'full_name': 'Casa do Benfica Santarém',
  'image': 'imgClubs/casa_benfica_santarem.jpg'},
 'CPMC': {'full_name': 'Casa Povo Miranda Corvo',
  'image': 'imgClubs/casa_povo_miranda_corvo.png'},
 'NSCM': {'full_name': 'Nú

In [6]:
# Define the HTML template for club and athlete information
template = """
<!DOCTYPE html>
<html>
<head>
    <style>
        table {{
            border-collapse: collapse;
            width: 100%;
            margin-bottom: 50px;
        }}
        th, td {{
            border: 1px solid black;
            text-align: center;
        }}
        th {{
            background-color: #f2f2f2;
            font-size: 15px;
        }}
        .td_name {{
            //text-align: left;
            font-size: 13px;
        }}
        .width_fix {{
            width: 90px;
        }}
        img {{
            max-width: 100px;
            max-height: 100px;
        }}
        .checkbox {{
            font-size: 24px;
        }}
    </style>
</head>
<body>
    <table>
        <tr>
            <th>{emblem}</th>
            <th colspan="8" style="text-align: left; padding-left: 12px; font-size: 19px;">{fullname}</th>
        </tr>
        <tr>
            <th>Nome do Atleta</th>
            <th style="width: 120px">Categoria de Inscrição</th>
            <th class="width_fix">Presente</th>
            <th class="width_fix">Peso Atual</th>
            <th class="width_fix">Mantém Categoria</th>
            <th class="width_fix">Sobe Categoria</th>
            <th class="width_fix">Desce Categoria</th>
            <th class="width_fix">Nova Categoria</th>
        </tr>
        {rows}
    </table>
</body>
</html>
"""

```shell
sudo chmod 0700 /run/user/1000/
```

In [7]:
# Function to generate PDF for each club
def generate_club_pdf(club_athletes, filename):
    
    options = {
        'encoding': "UTF-8",
        'page-size': 'A4',
        'orientation': 'Portrait',
        'margin-top': '0.75in',
        'margin-right': '0.5in',
        'margin-bottom': '0.75in',
        'margin-left': '0.5in',
    }

    pdf_content = []
    for club, athletes in club_athletes.items():
        # Replace club initials with full names
        club_info = clubs_dict.get(club)
        if club_info:
            club_name = club_info['full_name']
            if club_info['image'] != '':
                club_image_path = club_info['image']
            else:
                club_image_path = "imgClubs/placeholder.jpg"
        else:
            club_name = club  # Use initials if full name not found
            if club_info['image'] != '':
                club_image_path = club_info['image']
            else:
                club_image_path = "imgClubs/placeholder.jpg"

        # Convert image to base64 encoding
        img_data = ''
        if club_image_path is not None:
            try:
                with open(club_image_path, 'rb') as f:
                    img_data = base64.b64encode(f.read()).decode('utf-8')
                    img_tag = f'<img src="data:image/jpeg;base64,{img_data}">' if img_data else ''
                    emblem = f"{img_tag}"
            except FileNotFoundError:
                # Handle the case where the image file is not found
                print(f"Warning: Image file '{club_image_path}' not found for club '{club_name}'")
                club_image_path = None  # Set club_image_path to None to prevent further processing

        fullname = f"{club_name}"

        # Generate HTML for club and athlete information
        rows_html = ""
        for _, row in athletes.iterrows():
            rows_html += f"<tr> <td class=\"td_name\">{row['NAME']}</td> <td>{row['WGT_CAT']}</td> <td class=\"checkbox\">&#9744;</td> <td>_____</td> <td class=\"checkbox\">&#9744;</td> <td class=\"checkbox\">&#9744;</td> <td class=\"checkbox\">&#9744;</td> <td>_____</td></tr>"
        html_content = template.format(emblem = emblem, fullname= fullname, rows=rows_html)
        pdf_content.append(html_content)

    # Convert HTML to PDF
    pdfkit.from_string(''.join(pdf_content), filename, options=options)

# Generate PDF for masculine athletes
generate_club_pdf(masculine_club_athletes, f'Output/{tournament_code}_masculinos.pdf')

# Generate PDF for feminine athletes
generate_club_pdf(feminine_club_athletes, f'Output/{tournament_code}_femininos.pdf')